
discord music player?

export TOKENPATH=/Users/briancullinan/.credentials/discord-bot7.txt && export DEFAULT_APPLICATION=1335769252409380884 && export SECRETPATH=/Users/briancullinan/.credentials/discord7.txt && npm run import -- "discord music player"


In [ ]:

const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
const {discordExpress, closeExpress} = importer.import('discord music player server')

const ALL_COMMANDS = [
  'browse',
]

// bot commands using new API, same names as above but lower-case
async function playerCommands(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)

  interactionsCommands['startActivity'] = discordExpress
  interactionsCommands['endActivity'] = closeExpress

  cmdDef = {
    'name': 'browse',
    'description': 'Launch the music player',
    'type': 4,
    'handler': 2,
  }
  if(!commands.includes('browse')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'browse')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }

  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

module.exports = playerCommands





discord music player server?


In [ ]:
const {handleDirectory, BASE_DIRECTORY} = importer.import('node express directory handler')
const {DEFAULT_APPLICATION} = importer.import('discord configuration')
const getToken = importer.import('discord express token endpoint')
const url = require('url');
const path = require('path')

const INSTANCES = {}
const express = require('express')
const cors = require('cors')
const cookieParser = require('cookie-parser');

const app = express()

const BASE_URI = `https://${DEFAULT_APPLICATION}.discordsays.com/.proxy/`

process.env['NODE_TLS_REJECT_UNAUTHORIZED'] = 0;

app.use(cors({
  origin: '*' 
}))
app.use(express.json())
app.use(cookieParser())
app.post('/api/token', getToken)
app.post('/register', registerInstance)
app.get('*', handleDirectory)
app.use('/Music', express.static(path.join(BASE_DIRECTORY, 'Music')))
app.get('/amplitude.js', (req, res) => {
  return res.sendFile(path.join(__dirname, '../amplitude.js'))
})
app.get('/amplitude.css', (req, res) => {
  return res.sendFile(path.join(__dirname, '../amplitude.css'))
})

// TODO: share the rest of the files but only with logged in user
// return express.static(path.join(__dirname, 'public'));

let server

async function registerInstance(req, res, next) {
  var url_parts = url.parse(req.url, true);
  var query = url_parts.query;

  if(typeof query['launch_id'] != 'undefined' 
    && typeof query['frame_id'] != 'undefined' 
    && typeof query['user_id'] != 'undefined'
  ) {
    //let instance = await getInstance(query['instance_id'])
    //console.log(instance)
    if(typeof INSTANCES[query['launch_id']] == 'undefined') {
      INSTANCES[query['launch_id']] = []
    }
    INSTANCES[query['launch_id']].push([query['user_id'], query['frame_id']])
    console.log(INSTANCES)
    //console.log(req.cookies.discord_bot_expiry)
    let expiry = new Date(parseInt(query['expiry']))

    //authenticated = true
    // TODO: cookies
    res.cookie('discord_bot_session', query['user_id'], { 
      expires: expiry
    })
    res.cookie('discord_bot_frame', query['frame_id'], { 
      expires: expiry
    })
  }

  return res.send('')
}

async function discordExpress(activity) {
  if(!express) {
    return false
  }
  
  //process.env['NODE_TLS_REJECT_UNAUTHORIZED'] = 0;

  if(activity.participants.length == 0 && server) {
    server.close()
    server = null
  }

  // TODO: check if user was removed
  if(typeof INSTANCES[activity['instance_id']] == 'undefined') {
    INSTANCES[activity['instance_id']] = []
  }
  INSTANCES[activity['instance_id']] = INSTANCES[activity['instance_id']].filter(i => activity.participants.includes(i[0]))

  if(activity.length > 1 || server) {
    return
  }

  return (server = await app.listen(3001, () => {
    console.log('Listening on: ', 3001)
  }))
}

function closeExpress(activity) {
  if(activity) {
    if(typeof INSTANCES[activity['instance_id']] != 'undefined') {
      INSTANCES[activity['instance_id']] = INSTANCES[activity['instance_id']].filter(i => activity.participants.includes(i[0]))
    }
    return
  }
  if(server) {
    server.close()
    server = null
  }
}

module.exports = {
  discordExpress,
  closeExpress,
  INSTANCES
}


